In [ ]:
import os
import torch

from testdataset import get_loader
from architectures import UNet
from testoperation import run

## 1. Global Variables and Backend Fixation

In [ ]:
test_path = './_data/test.pkl'
save_dir = './_saved_models'
result_dir = './_results'

torch.backends.cudnn.deterministic = True 
torch.backends.cudnn.benchmark = False

## 2. GPU Mounting

In [ ]:
def gpu_manager(model):
    device_cnt = torch.cuda.device_count()
    if device_cnt > 0:
        if device_cnt == 1:
            print('Only 1 GPU is available.')
        else:
            print(f"{device_cnt} GPUs are available.")
            model = torch.nn.DataParallel(model)
        model = model.cuda()
    else:
        print('Only CPU is available.')
        
    return model

## 3. Configuration

### 3.1. About Padding:
when using valid padding, the output size may not match input size, thus zero padding the input is needed,

the sizes should be calculated in advance based on the network structure, in our case (default UNet):
- `input_size = 204`, 
- `pad_size = (input_size - output_size)//2`

if same padding then `pad_size = 0`

### 3.2. UNet Args:
- **in_channels (int)**: number of input channels
- **n_classes (int)**: number of output channels
- **depth (int)**: depth of the network
- **wf (int)**: number of filters in the first layer is `2**wf`
- **padding (bool)**: if True, apply padding such that the input shape is the same as the output. This may introduce artifacts
- **batch_norm (bool)**: Use BatchNorm after layers with an activation function
- **up_mode (str)**: one of 'upconv' or 'upsample'. 'upconv' will use transposed convolutions for learned upsampling. 'upsample' will use bilinear upsampling.

In [ ]:
resize = {
    'pad_size': 46,
    'output_size': 112,## plz use even single values, odd size and rectangles are not taken care of!
}

model_params = {
    'in_channels':1,
    'n_classes':2,
    'depth':4,## 4 at maximum
    'wf':5,
    'padding':False,
    'batch_norm':False,
    'up_mode':'upconv',
}

In [ ]:
run_params = {
    'dataloader': get_loader(test_path, resize),
    'model': gpu_manager(UNet(**model_params)),
    'ckpt_path': os.path.join(save_dir, 'checkpoint33.tar.gz')
}

In [ ]:
soft_preds = run(**run_params)

In [ ]:
import pickle
import gzip

def save_zipped_pickle(obj, filename):
    with gzip.open(filename, 'wb') as f:
        pickle.dump(obj, f, 2)

In [ ]:
save_zipped_pickle(soft_preds, os.path.join(result_dir, 'soft_pred_ckpt33.pkl'))